In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import time

In [2]:
data = np.load('/home/yxf118/ROS_car/vehicle_data/cnn_features.npz')
#data = np.load('/home/yxf118/ROS_car/vehicle_data/dataset.npz')
X_train, y_train = data['X_train'], data['y_train']
X_test, y_test = data['X_test'], data['y_test']

In [3]:
# Parameters
learning_rate = 0.001
batch_size = 200
epochs = 40
n_input = 64 # data features(64))
n_steps = 10 # timesteps
n_seqs = batch_size/n_steps
n_hidden = 200 # hidden layer num
n_layers = 2
n_classes = 7 
keep_prob = 0.5 

In [4]:
np.shape(X_train)

(3025, 64)

In [5]:
np.shape(y_train)

(3025, 7)

In [6]:
X_train[0,:]

array([  1.39717662,   0.        ,   0.        ,   0.        ,
         0.        ,  11.44458485,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         4.24545288,   0.        ,   4.41768837,   0.        ,
         7.37591171,   0.        ,   5.9482255 ,   0.        ,
         0.        ,  18.33749771,   0.        ,   0.60286605,
         0.        ,   0.        ,   7.91732502,  21.76796341,
         0.85880768,  10.54665375,   9.31004524,   0.41251412,
         0.        ,   0.        ,   0.        ,   0.        ,
         9.73074627,   0.        ,   0.37255639,   4.39681864,
         0.        ,   0.        ,   0.        ,  11.11513519,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  11.45240688,   0.        ,
         2.02771354,   0.        ,   0.10151491,   0.        ,
         0.        ,   4.74020338,   1.10240889,   0.  

In [7]:
y_train[0,:]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.])

In [8]:
def get_batches(X,Y, n_seqs, n_steps,n_input,n_classes):
    '''
    slice the mini-batches
    
    X: X_input, to be sliced
    n_seqs: number of sequences
    n_steps: num of steps (in time)
    n_inputs: input features
    n_classes: output classes
    '''
    batch_size = n_seqs * n_steps
    n_batches = int(len(X) / batch_size)
    # keep only integer batches
    X = X[:int(batch_size * n_batches)]
    Y = Y[:int(batch_size * n_batches)]
    
    # reshape
    X = X.reshape((int(n_seqs), -1, n_input))
    Y = Y.reshape((int(n_seqs), -1, n_classes))
    
    for n in range(0, X.shape[1], n_steps):
        # inputs
        x = X[:, n:n+n_steps,:]
        # targets
        y = np.zeros((x.shape[0],x.shape[1],n_input))
        y = Y[:, n:n+n_steps,:]
        yield x, y

In [9]:
batches = get_batches(X_train,y_train,10,10,64,7)
x, y = next(batches)

In [10]:
np.shape(x)

(10, 10, 64)

In [11]:
def build_inputs(num_seqs, num_steps,n_classes):
    '''
    building the input layer
    
    num_seqs: number of sequences in every batch (1st dimension)
    num_steps: number of time steps in each sequence (2nd dimension)
    '''
#    inputs = tf.placeholder(tf.float32, shape=(None, num_steps, n_input), name='inputs')
#    targets = tf.placeholder(tf.int64, shape=(None, num_steps, n_classes), name='targets')
    inputs = tf.placeholder(tf.float32, shape=(num_seqs, num_steps, n_input), name='inputs')
    targets = tf.placeholder(tf.float32, shape=(num_seqs, num_steps, n_classes), name='targets')

    # add the keep_prob
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [12]:
def build_lstm(lstm_size, num_layers, batch_size,num_steps, keep_prob):
    ''' 
    building the lstm layer
        
    keep_prob: dropout keep probability
    lstm_size: number of hidden units in lstm layer
    num_layers: number of lstm layers
    batch_size: batch_size

    '''
    # build a basic lstm unit
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # adding dropout
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    lstm2 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop2 = tf.contrib.rnn.DropoutWrapper(lstm2, output_keep_prob=keep_prob)

    stack_rnn = [drop]
    for _ in range(num_layers-1):
        stack_rnn.append(drop2)


    # stack (changed in TF 1.2)   
    cell = tf.contrib.rnn.MultiRNNCell(stack_rnn, state_is_tuple = True)
#    cell = tf.contrib.rnn.MultiRNNCell([drop for _ in range(num_layers)])

    initial_state = cell.zero_state(int(batch_size/num_steps), tf.float32)
    
    return cell, initial_state

In [13]:
def build_output(lstm_output, in_size, out_size):
    ''' 
    building the output layer
        
    lstm_output: the output of the lstm layer
    in_size: lstm layer reshaped size
    out_size: softmax layer size
    
    '''

    # concate lstm output according to the output sequence，
    # [[1,2,3],[7,8,9]] tf.concat -> [1,2,3,7,8,9]
    seq_output = tf.concat(lstm_output, axis=1) # tf.concat(concat_dim, values)
    # reshape
    x = tf.reshape(seq_output, [-1, in_size])
    
    # fully connect lstm to softmax
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # compute logits
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # softmax return
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [14]:
def build_loss(logits, targets, lstm_size, num_classes):
    '''
    compute loss using logits and targets
    
    logits: fully connected layer output（before softmax）
    targets: targets
    lstm_size: lstm_size
    num_classes: class size
        
    '''
#    # One-hot coding
#    y_one_hot = tf.one_hot(targets, num_classes)
#    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
#    
#    # Softmax cross entropy loss
#    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
#    loss = tf.reduce_mean(loss)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets))
    return loss

In [15]:
def build_optimizer(loss, learning_rate, grad_clip):
    
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [16]:
class LSTM_RNN:
    
    def __init__(self, num_classes, batch_size=100, num_steps=10, 
                       lstm_size=128, num_layers=n_layers, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # if sampling is True，use SGD, only 1 sample
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # input layer
        self.inputs, self.targets, self.keep_prob = build_inputs(n_seqs, num_steps, num_classes)

        # LSTM layer
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, n_steps,self.keep_prob)

#        # one-hot coding for inputs
#        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # running the RNN
        outputs, state = tf.nn.dynamic_rnn(cell, self.inputs, initial_state=self.initial_state)
        self.final_state = state
        
        # predicting the results
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
        
        correct_predictions = tf.equal(tf.argmax(self.prediction, 1),
                                       tf.reshape(tf.argmax(self.targets, 2), [-1]))
        self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"))

In [17]:
model = LSTM_RNN(y_train.shape[1], batch_size=batch_size, num_steps=n_steps,
                lstm_size=n_hidden, num_layers=2, learning_rate=learning_rate)

In [18]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
        
    for e in tqdm(range(epochs), desc="\nTraining progress"):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(X_train,y_train,n_seqs,n_steps,n_input,n_classes):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            # control the print lines
            if counter % 100 == 0:
                print('\n',
                      'Epoches: {}/{}... '.format(e+1, epochs),
                      'Training Steps: {}... '.format(counter),
                      'Training Loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
                counter2 = 0
                test_accuracy = 0
                for X_test_lstm, y_test_lstm in get_batches(X_test,y_test,n_seqs,n_steps,n_input,n_classes):
                    counter2 += 1
                    feed = {model.inputs: X_test_lstm,
                            model.targets: y_test_lstm,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    accuracy, = sess.run([model.accuracy], feed_dict=feed)
                    test_accuracy += accuracy
                test_accuracy = test_accuracy/counter2
                print('\n', 'Test accuracy: {:.4f}... '.format(test_accuracy))

    saver.save(sess, "/home/yxf118/ROS_car/vehicle_data/lstm_model/checkpoints/i{}_l{}.ckpt".format(counter, n_hidden))


Training progress:  15%|█▌        | 6/40 [00:04<00:24,  1.40it/s]


 Epoches: 7/40...  Training Steps: 100...  Training Loss: 0.1244...  0.0394 sec/batch

 Test accuracy: 0.6600... 



Training progress:  32%|███▎      | 13/40 [00:08<00:15,  1.72it/s]


 Epoches: 14/40...  Training Steps: 200...  Training Loss: 0.1144...  0.0381 sec/batch

 Test accuracy: 0.7900... 



Training progress:  50%|█████     | 20/40 [00:12<00:11,  1.73it/s]

...


 Epoches: 27/40...  Training Steps: 400...  Training Loss: 0.0438...  0.0387 sec/batch

 Test accuracy: 0.8200... 



Training progress:  82%|████████▎ | 33/40 [00:19<00:04,  1.73it/s]


 Epoches: 34/40...  Training Steps: 500...  Training Loss: 0.0872...  0.0559 sec/batch

 Test accuracy: 0.8200... 



Training progress: 100%|██████████| 40/40 [00:23<00:00,  1.74it/s]


 Epoches: 40/40...  Training Steps: 600...  Training Loss: 0.0295...  0.0363 sec/batch

 Test accuracy: 0.8850... 


In [19]:
#%% inference

checkpoint = tf.train.latest_checkpoint('/home/yxf118/ROS_car/vehicle_data/lstm_model/checkpoints/')
model = LSTM_RNN(y_train.shape[1], batch_size=batch_size, num_steps=n_steps,
                lstm_size=n_hidden, num_layers=n_layers, learning_rate=learning_rate, 
                sampling=False)
saver = tf.train.Saver()
with tf.Session() as sess:
    # load the model and restoring
    saver.restore(sess, checkpoint)
    new_state = sess.run(model.initial_state)
    
    counter = 0
    test_accuracy = 0
    for X_test_lstm, y_test_lstm in get_batches(X_test,y_test,n_seqs,n_steps,n_input,n_classes):
        counter += 1
        feed = {model.inputs: X_test_lstm,
                model.targets: y_test_lstm,
                model.keep_prob: 1.,
                model.initial_state: new_state}
        accuracy, = sess.run([model.accuracy], feed_dict=feed)
        test_accuracy += accuracy
    test_accuracy = test_accuracy/counter
    print('Test accuracy: {:.4f}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from /home/yxf118/ROS_car/vehicle_data/lstm_model/checkpoints/i600_l200.ckpt
Test accuracy: 0.9250
